In [1]:
project: getenv `project_id
csbucketname: getenv `csbucketname

# From BigQuery to kdb

In [2]:
// extract from BigQuery to Cloud Storage
system "bq extract --destination_format NEWLINE_DELIMITED_JSON ", project, ":bqkdb.allBQSimpleTypes gs://", csbucketname, "/allBQSimpleTypes.json"

Waiting on bqjob_r449c22e13fbc4e13_0000016e4638ce7d_1 ... (0s) Current status: DONE   

""


In [3]:
csfilename: "gs://", csbucketname, "/allBQSimpleTypes.json"

In [ ]:
// Copy from Cloud Storage to local box
system "gsutil cp ", csfilename, " /tmp/"

Copying gs://storagebodon/allBQSimpleTypes.json...


## Manual type conversion
* string, boolean and float types are properly handled
* other types need to be casted manually
* timestamps needs minor conversion, chopping timezone information off


In [ ]:
decode: {[fM; j] k: .j.k j; key[k]!fM[key k]@'value k}

In [ ]:
formatterMap: (enlist `)!enlist (::)   // default formatter: leave unchanged

In [ ]:
formatterMap[`int]: "I"$
formatterMap[`ts]: 	{"P"$-4 _ x}
formatterMap[`date]: "D"$
formatterMap[`time]: "T"$
formatterMap[`dt]: "P"$

In [ ]:
t: formatterMap decode/: read0 hsym `$"/tmp/allBQSimpleTypes.json"

In [ ]:
t

In [ ]:
meta t

# From kdb to BigQuery

Let us save the fixed kdb table to JSON

In [ ]:
save `:/tmp/t.json

In [ ]:
system "bq load --autodetect --source_format NEWLINE_DELIMITED_JSON bqkdb.allBQSimpleTypes_json /tmp/t.json"

We can see that the **column orders do not match**

In [ ]:
system "bq show bqkdb.allBQSimpleTypes_json"

## Cleanup

In [ ]:
// Cloud Storage
system "gsutil rm ", csfilename

In [ ]:
// local files
system "rm /tmp/t.json"

In [ ]:
// BigQuery table
system "bq rm -f bqkdb.allBQSimpleTypes_json"